In [1]:
from matplotlib import pyplot as plt
import polars as pl
import seaborn as sns
import japanize_matplotlib

from common import *

In [2]:
N_COMPONENTS = 50
TEST_SIZE = 0.2
VAL_SIZE = 0.2
BAG_SIZE = 4

In [3]:
X_train, X_val, X_test, y_train, y_val, y_test = load_mnist_pca_train_test_val(
    n_components=N_COMPONENTS, test_size=TEST_SIZE, val_size=VAL_SIZE
)

Files already downloaded
Files already downloaded


In [4]:
X_train, y_train = rewrite_label_with_binary_setting(X_train, y_train)
X_val, y_val = rewrite_label_with_binary_setting(X_val, y_val)
X_test, y_test = rewrite_label_with_binary_setting(X_test, y_test)

In [5]:
def train_lgbm_with_custom_loss(X_train, X_valid, y_train, y_valid):

    """Train LightGBM with custom loss function."""
    train_data = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
    valid_data = lgb.Dataset(X_valid, label=y_valid, free_raw_data=False)

    params = {
        "objective": "binary",
        "verbose": -1,
        "learning_rate": 0.1,
        "num_boost_round": 200,
    }
    valid_accuracies = []

    def record_accuracies(p: lgb.Booster, train_data: lgb.Dataset, valid_data: lgb.Dataset):
        valid_pred = (p.predict(valid_data.data) > 0.5).astype(int)
        valid_acc = accuracy_score(valid_data.label, valid_pred)
        valid_accuracies.append(valid_acc)
        print(valid_acc)
    
    gbm = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        callbacks=[lambda p: record_accuracies(p.model, train_data, valid_data)]
    )
    return gbm, valid_accuracies

In [6]:
gbm, valid_accuracies =train_lgbm_with_custom_loss(X_train, X_val, y_train, y_val)
y_pred = gbm.predict(X_val)
print("Accuracy: {}".format(accuracy_score(y_val, (y_pred > 0.5).astype(int))))

/Users/ryoma.kobayashi/gits/techblog_git/wsl_w_lgbm/.venv/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


0.8533333333333334
0.86025
0.8633333333333333
0.8683333333333333
0.87575
0.8825833333333334
0.8883333333333333
0.8975833333333333
0.8990833333333333
0.9051666666666667
0.9071666666666667
0.9095833333333333
0.9141666666666667
0.9173333333333333
0.9173333333333333
0.9191666666666667
0.92125
0.92325
0.92625
0.92725
0.9275
0.9280833333333334
0.93
0.93175
0.9326666666666666
0.9339166666666666
0.93425
0.9350833333333334
0.9364166666666667
0.9370833333333334
0.9381666666666667
0.9398333333333333
0.9396666666666667
0.9414166666666667
0.9421666666666667
0.9426666666666667
0.9441666666666667
0.9440833333333334
0.94525
0.9456666666666667
0.94625
0.9463333333333334
0.9469166666666666
0.9471666666666667
0.948
0.9485833333333333
0.94975
0.95
0.9500833333333333
0.9505
0.95125
0.95025
0.951
0.9524166666666667
0.9531666666666667
0.9536666666666667
0.9538333333333333
0.9544166666666667
0.9549166666666666
0.9554166666666667
0.9554166666666667
0.9558333333333333
0.9561666666666667
0.9568333333333333
0.957

In [7]:
np.save("sup_bin_valid_accuracies.npy", valid_accuracies)